# BlueSprig-CV 蓝枝简历 — 简单demo演示
## 1. 概述
    本demo演示了本组使用蓝心大模型，通过构建prompt和message，实现了部分最终产品功能的python代码演示

主要功能
1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式
2. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化
3. 根据规范化简历json，基于场景构建一场模拟面试，用语音播放题目，允许用户使用语音或文字回答问题，并在最终给出面试评分
    1. 面试场景可以分为：技术面试、产品面试、HR面试、技术+产品面试、技术+HR面试等等，后续可能根据不同行业标准进行调整，可以通过RAG减弱大模型幻觉现象，保证面试题更贴近真实面试，（本功能点因为时间问题当前demo并未实现）


## 2. 主要功能演示
### 1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式

In [1]:
import json

# 工具方法
from util.file_loader import load_file
from util.llm import format_resume


# 定义一个函数，用于读取txt文件
def read_txt(file_path):
    # 打开文件，以只读模式，编码格式为utf-8
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取文件内容，并去除首尾空格
        return file.read().strip()

In [2]:
# 读取简历文件
resume_path = 'assets/pdf/1.pdf'
content = load_file(resume_path)
content

'张文博\n意向岗位: 后端开发。\n            \n            \n            \n\n电话：15602032933\n邮箱：Hotcocoa.Canary@outlook.com。\n            \n            \n            \n\n性别：男\n年龄：21。\n            \n            \n            \n\n户籍：内蒙古自治区\n政治面貌：共青团员。\n            \n            \n            \n\n民族：汉。\n            \n            \n            \n\n求职意向。\n            \n            \n            \n\n意向岗位: 后端开发\n意向城市: 北京。\n            \n            \n            \n\n期望薪资: 8k-20k\n求职类型: 校招。\n            \n            \n            \n\n期望行业: 计算机软件\n当前状态: 月内到岗。\n            \n            \n            \n\n教育经历。\n            \n            \n            \n\n内蒙古农业大学。\n            \n            \n            \n\n2022.09 - 2025.07。\n            \n            \n            \n\n软件工程\n本科\n|。\n            \n            \n            \n\n荣誉/奖项：。\n            \n            \n            \n\n• 自治区励志奖学金（2023-10）。\n            \n            \n            \n\n• 多次获得校级一等优秀学生（2024-5，2024-11）。\n            \n            \n    

In [3]:
# 获取标准解析简历json格式
resume_format = read_txt('assets/txt/resume_format.txt')
response_content = format_resume(resume_format, content)
response_content

'{"job_intent_info":[{"job_intention":"后端开发","expected_salary":"8k-20k","preferred_city":"北京","job_industry":"计算机软件","job_type":"校招"}],"basic_info":{"name":"张文博","age":"21","sex":"男","native_place":"内蒙古自治区","education":"本科","if_party_membership":"共青团员","marital_status":"","phone_number":"15602032933","email":"Hotcocoa.Canary@outlook.com","if_fresh_graduate":"是","years_of_work_experience":"","expected_salary":""},"work_experience":[{"job_position":"大模型开发实习生","work_period":"2024.02 - 2024.10","company_name":"北京以塔科技有限公司","work_content":"• 智能简历解析系统开发。\\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\\n• Task：构建基于大模型的简历结构化处理与诊断系统。\\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\\n简历优化）。\\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\\n• RAG增强问答系统开发。\\n•  Context：构建基于pdf检索增强生成的问答系统。\\n• Task：实现文档智能解析与语义检索核心模块。\\n• Action：基于LangChain搭建处理流水线，设计混合检索策略：关键词匹配 + 语义检索（ChromaDB向量库），实\\n现上下文感知问答：通过Prompt工程优化大模型响应准确。\\n• Res

### 3. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化

#### 3.1. 简历优化版本

In [4]:
# 简历json数据
resume_json = json.loads(response_content)
resume_json

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'job_position': '大模型开发实习生',
   'work_period': '2024.02 - 2024.10',
   'company_name': '北京以塔科技有限公司',
   'work_content': '• 智能简历解析系统开发。\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\n• Task：构建基于大模型的简历结构化处理与诊断系统。\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\n简历优化）。\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\n• RAG增强问答系统开发。\n•  Context：构建基于pdf检索增强生成的问答系统。\n• Task：实现文档智能解析与语义检索核心模块。\n• Action：基于LangChain搭建处理流水

In [5]:
from util.llm import better_resume_by_module

work_experience_content = resume_json["work_experience"]
education_experience_content = resume_json["education_experience"]
project_experience_content = resume_json["project_experience"]

work_experience_format = read_txt("assets/txt/greater_resume_module/work_experience.txt")
education_experience_format = read_txt("assets/txt/greater_resume_module/education_experience.txt")
project_experience_format = read_txt("assets/txt/greater_resume_module/project_experience.txt")

work = better_resume_by_module(work_experience_format, work_experience_content, "工作经历")
education = better_resume_by_module(education_experience_format, education_experience_content, "教育经历")
project = better_resume_by_module(project_experience_format, project_experience_content, "项目经历")

project = json.loads(project)
work = json.loads(work)
education = json.loads(education)

resume_json["project_experience"] = project
resume_json["work_experience"] = work
resume_json["education_experience"] = education

resume_json

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'company': '北京以塔科技有限公司',
   'position': '大模型开发实习生',
   'location': '',
   'duration': '2024.02 - 2024.10',
   'responsibilities': [{'item': '智能简历解析系统开发',
     'description': '应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。通过采用大模型Prompt工程针对项目接口进行开发，完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化。'},
    {'item': 'RAG增强问答系统开发',
     'description': '构建基于pdf检索增强生成的问答系统，实现文档智能解析与语义检索核心模块。基于LangChain搭建处理流水线，设计混合检索策略：关键词匹配 + 语义检索（ChromaDB向量库），实现上下文感知问答：通过Prompt工程优化大模型响应准确。'},
    {'item': 'Dian项目后端架构与部署',
     

#### 3.2. 简历优化对话

In [6]:
# 获取原始简历
resume_json_old = json.loads(response_content)
resume_json_old

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'job_position': '大模型开发实习生',
   'work_period': '2024.02 - 2024.10',
   'company_name': '北京以塔科技有限公司',
   'work_content': '• 智能简历解析系统开发。\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\n• Task：构建基于大模型的简历结构化处理与诊断系统。\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\n简历优化）。\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\n• RAG增强问答系统开发。\n•  Context：构建基于pdf检索增强生成的问答系统。\n• Task：实现文档智能解析与语义检索核心模块。\n• Action：基于LangChain搭建处理流水

In [7]:
# 获取优化后的简历
resume_json_new = resume_json
resume_json_new

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'company': '北京以塔科技有限公司',
   'position': '大模型开发实习生',
   'location': '',
   'duration': '2024.02 - 2024.10',
   'responsibilities': [{'item': '智能简历解析系统开发',
     'description': '应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。通过采用大模型Prompt工程针对项目接口进行开发，完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化。'},
    {'item': 'RAG增强问答系统开发',
     'description': '构建基于pdf检索增强生成的问答系统，实现文档智能解析与语义检索核心模块。基于LangChain搭建处理流水线，设计混合检索策略：关键词匹配 + 语义检索（ChromaDB向量库），实现上下文感知问答：通过Prompt工程优化大模型响应准确。'},
    {'item': 'Dian项目后端架构与部署',
     

In [8]:
# 介于jupyter的限制，这里采用的预设好的用户问题对话，我们也制作的终端对话的脚本，请参照show_main.py

### 4. 根据规范化简历json，基于场景构建一场模拟面试，用语音播放题目，允许用户使用语音或文字回答问题，并在最终给出面试评分
#### 4.1. 模拟面试

In [9]:
# 获取简历json
resume_json

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'company': '北京以塔科技有限公司',
   'position': '大模型开发实习生',
   'location': '',
   'duration': '2024.02 - 2024.10',
   'responsibilities': [{'item': '智能简历解析系统开发',
     'description': '应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。通过采用大模型Prompt工程针对项目接口进行开发，完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化。'},
    {'item': 'RAG增强问答系统开发',
     'description': '构建基于pdf检索增强生成的问答系统，实现文档智能解析与语义检索核心模块。基于LangChain搭建处理流水线，设计混合检索策略：关键词匹配 + 语义检索（ChromaDB向量库），实现上下文感知问答：通过Prompt工程优化大模型响应准确。'},
    {'item': 'Dian项目后端架构与部署',
     

In [10]:
from util.llm import generate_interview_questions

# 生成面试题列表
interview_questions = generate_interview_questions(resume_json)
questions_list = json.loads(interview_questions)["questions"]
questions_list

['请介绍一下你在北京以塔科技有限公司实习期间所负责的智能简历解析系统开发项目。',
 '在RAG增强问答系统开发中，你遇到了哪些挑战？',
 '在Dian项目后端架构与部署中，你是如何实现基础环境隔离的？',
 '请介绍一下你在Note-taking系统开发中的具体职责。',
 '在AI-云寻项目中，你是如何实现知识图谱可视化开发的？',
 '在自然语言处理（NLP）模块中，你是如何实现英文实体识别的？',
 '请介绍一下你在内蒙古农业大学的学习经历，以及你所参与的校园活动。',
 '你在技术技能方面有哪些特长？',
 '请介绍一下你在普通话二级甲等、全国大学生数学建模竞赛等竞赛中的表现。',
 '你未来的职业规划是什么？']

In [11]:
from util.llm import generate_questions_audio

# 生成面试题语音
questions_audio_list = generate_questions_audio(questions_list)
questions_audio_list

['assets/audio\\short_audio_synthesis_jovi_yige_20250501225146.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225148.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225151.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225153.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225156.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225200.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225203.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225204.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225207.wav',
 'assets/audio\\short_audio_synthesis_jovi_yige_20250501225209.wav']

#### 4.2. 面试评分

In [12]:
from util.llm import audio2text

# 处理用户答案，这里还是采用预设的问题和结果进行演示
questions_list = ['请介绍一下你在北京以塔科技有限公司的工作经历，以及你在大模型开发实习生职位上的主要职责。',
                  '在AI-云寻项目中，你负责了哪些具体的技术开发和系统架构设计？',
                  '请谈谈你在内蒙古农业大学的学习经历，以及你在软件工程专业的学术成果。',
                  '你在项目开发中使用了哪些编程语言和技术框架？',
                  '请描述一下你在技术团队中的角色和职责，以及如何与团队成员协作完成项目。',
                  '你在技术解决方案设计和实施过程中遇到过哪些挑战？如何解决的？',
                  '请谈谈你在技术项目管理方面的经验和方法。',
                  '你在技术团队中的沟通和协作能力如何？',
                  '请描述一下你在技术学习和自我提升方面的计划和实践。',
                  '你对未来技术发展趋势有何看法？对加入我们公司的期望和计划是什么？']
user_answers = [
    {
        "answer": "我在北京以塔科技有限公司担任大模型开发实习生期间，主要负责简历解析系统和RAG增强问答系统的研发。我的工作包括：基于大模型的Prompt工程开发，实现简历结构化处理、诊断和优化；使用LangChain构建问答系统处理流水线，结合关键词匹配与ChromaDB语义检索，实现上下文感知问答；熟悉大模型返回值的解析与JSON格式规范化，优化交互体验；参与项目的前后端部署工作，包括使用Nginx进行Flask和Next.js服务集成，以及在Ubuntu服务器上部署数据库与API服务。这段实习经历不仅锻炼了我对大模型应用的掌握能力，还加深了我对完整工程闭环的理解，包括从本地开发到线上部署的全过程。",
        "type": "text"
    },
    {
        "answer": "在AI-云寻项目中，我主要负责了系统架构设计和核心模块开发：系统架构：采用前后端分离架构，前端用 Vue 3 + Element Plus + ECharts，后端使用 Spring Boot 提供 RESTful API，数据库采用 Neo4j 图数据库。知识图谱可视化：设计节点（如论文、作者、机构）和关系结构，使用 ECharts 力导向图实现动态渲染，支持分页加载优化性能。NLP模块：集成 Apache OpenNLP，实现英文实体识别和结构化数据抽取，自动生成 Cypher 语句导入 Neo4j。数据导入导出：支持Excel批量导入/导出，提升数据处理效率。智能问答接口：结合 NLP 与图数据库，实现语义化的知识问答功能。这个项目强化了我对图数据库、NLP与可视化技术融合的理解和实战能力。",
        "type": "text"
    },
    {
        "answer": "assets/audio/面试问题3.m4a",
        "type": "audio"
    },
    {
        "answer": "我常用的编程语言包括：Java、Python、JavaScript。使用过的技术栈包括：后端：Spring Boot、Flask、MyBatis、JWT；前端：Vue 3、Element Plus、ECharts、PDF.js；数据库：MySQL、Neo4j、ChromaDB（向量库）；AI/NLP：LangChain、OpenNLP、大语言模型 Prompt 编程；部署与环境管理：Nginx、Linux(Ubuntu)、Git、环境变量配置。",
        "type": "text"
    },
    {
        "answer": "我曾在多个项目中担任核心开发者或项目经理角色：例如在“智慧农业系统”中担任项目经理，负责团队分工、进度安排和代码审核，协调6人团队协同开发；在“Note-taking系统”中，负责核心模块的设计与开发，并主导从开发到部署的全流程；采用多人协作工具如Git进行版本控制，配合Code Review流程，保持代码一致性；注重文档编写与沟通效率，确保团队对项目目标理解一致。",
        "type": "text"
    },
    {
        "answer": "在构建大模型简历诊断系统时，遇到以下挑战：大模型输出不稳定：返回结构不一致，导致后端解析失败。→ 解决方法：通过Prompt工程优化输出格式，并编写容错解析器，提升系统鲁棒性。PDF简历图片信息提取复杂：OCR准确率不高，信息丢失。→ 解决方法：引入图片识别模型与关键字段规则匹配结合，提升识别质量。向量检索系统响应慢：查询时延迟高→ 解决方法：对语义检索流程进行缓存策略优化，减少重复查询。",
        "type": "text"
    },
    {
        "answer": "我在“智慧农业系统”中担任项目经理，采用以下项目管理方式：使用甘特图制定开发计划与里程碑；每日召开短会复盘进展，快速发现问题；拆分任务模块，明确责任人；使用Git进行版本控制，保持开发节奏同步；项目尾声组织功能回归测试和Bug修复。通过有效管理，我带领团队按时高质量完成了项目交付。",
        "type": "text"
    },
    {
        "answer": "我擅长团队内外的技术沟通：善于将复杂技术方案用图解和比喻向非技术同学说明；前端、算法、产品岗位保持紧密联动，确保理解一致；常使用协作平台（如飞书、GitHub Issue）明确问题与进度。团队成员普遍反馈我沟通清晰、有条理，能在关键时刻推动问题解决。",
        "type": "text"
    },
    {
        "answer": "我保持每周定期学习，内容包括新技术文档、开源项目源码阅读；每季度完成一个小项目或参加技术竞赛检验成果；订阅AI/后端领域的技术博客、GitHub Star优秀项目；近期重点关注RAG系统、LangChain、LLM微调与Agent开发相关知识；计划后续深入参与开源社区，锻炼工程协作能力。",
        "type": "text"
    },
    {
        "answer": "我认为未来技术将围绕“大模型 + 垂直场景”的融合展开，特别是在RAG、Agent、MaaS（Model-as-a-Service）方向。以Prompt为接口的系统工程能力会变得非常重要。加入贵公司后，我期望能：深入参与大模型产品研发，提升Prompt工程与后端集成能力；在实际项目中参与高并发、高可用系统的构建；向经验丰富的工程师学习，快速成长为能独立负责模块设计与实现的工程人才。",
        "type": "text"
    }
]

answers_list = []
answers_result = []

for i in range(len(user_answers)):
    if user_answers[i]["type"] == "text":
        answers_list.append(user_answers[i]["answer"])
    else:
        answers_list.append(audio2text(user_answers[i]["answer"]))

for i in range(len(questions_list)):
    answers_result.append({
        "question": questions_list[i],
        "answer": answers_list[i]
    })

answers_result

Uploaded slice 1/1
Progress: 0%
Progress: 0%
Progress: 100%


[{'question': '请介绍一下你在北京以塔科技有限公司的工作经历，以及你在大模型开发实习生职位上的主要职责。',
  'answer': '我在北京以塔科技有限公司担任大模型开发实习生期间，主要负责简历解析系统和RAG增强问答系统的研发。我的工作包括：基于大模型的Prompt工程开发，实现简历结构化处理、诊断和优化；使用LangChain构建问答系统处理流水线，结合关键词匹配与ChromaDB语义检索，实现上下文感知问答；熟悉大模型返回值的解析与JSON格式规范化，优化交互体验；参与项目的前后端部署工作，包括使用Nginx进行Flask和Next.js服务集成，以及在Ubuntu服务器上部署数据库与API服务。这段实习经历不仅锻炼了我对大模型应用的掌握能力，还加深了我对完整工程闭环的理解，包括从本地开发到线上部署的全过程。'},
 {'question': '在AI-云寻项目中，你负责了哪些具体的技术开发和系统架构设计？',
  'answer': '在AI-云寻项目中，我主要负责了系统架构设计和核心模块开发：系统架构：采用前后端分离架构，前端用 Vue 3 + Element Plus + ECharts，后端使用 Spring Boot 提供 RESTful API，数据库采用 Neo4j 图数据库。知识图谱可视化：设计节点（如论文、作者、机构）和关系结构，使用 ECharts 力导向图实现动态渲染，支持分页加载优化性能。NLP模块：集成 Apache OpenNLP，实现英文实体识别和结构化数据抽取，自动生成 Cypher 语句导入 Neo4j。数据导入导出：支持Excel批量导入/导出，提升数据处理效率。智能问答接口：结合 NLP 与图数据库，实现语义化的知识问答功能。这个项目强化了我对图数据库、NLP与可视化技术融合的理解和实战能力。'},
 {'question': '请谈谈你在内蒙古农业大学的学习经历，以及你在软件工程专业的学术成果。',
  'answer': '我在内蒙古农业大学主修软件工程系统学习了软件开发身份周期、数据结构与算法、数据库原理、外部开发等课程期间，我积极参与竞赛与项目实践荣获icpc中国西部银奖、丝绸之路邀请赛银奖、篮球杯、省赛二等奖、数学剑魔、内蒙古散益区三等奖。  多次获得校一等奖学金和励志奖学金。 此外，我还担任计

In [13]:
# 面试结果
from util.llm import interview_result

interview_result_format = read_txt("assets/txt/interview_result_format.txt")
interview_result = interview_result(interview_result_format, resume_json, answers_result)
interview_result

'{"candidate": {"name": "张文博", "position": "Java后端工程师", "interview_date": "2025-05-01"}, "result": [{"question": "请介绍一下你在北京以塔科技有限公司的工作经历，以及你在大模型开发实习生职位上的主要职责。", "answer": "我在北京以塔科技有限公司担任大模型开发实习生期间，主要负责简历解析系统和RAG增强问答系统的研发。我的工作包括：基于大模型的Prompt工程开发，实现简历结构化处理、诊断和优化；使用LangChain构建问答系统处理流水线，结合关键词匹配与ChromaDB语义检索，实现上下文感知问答；熟悉大模型返回值的解析与JSON格式规范化，优化交互体验；参与项目的前后端部署工作，包括使用Nginx进行Flask和Next.js服务集成，以及在Ubuntu服务器上部署数据库与API服务。这段实习经历不仅锻炼了我对大模型应用的掌握能力，还加深了我对完整工程闭环的理解，包括从本地开发到线上部署的全过程。", "score": 9, "comments": "展示了对大模型开发的专业知识和实际工作经验，表述清晰，展现了良好的技术能力。"}, {"question": "在AI-云寻项目中，你负责了哪些具体的技术开发和系统架构设计？", "answer": "在AI-云寻项目中，我主要负责了系统架构设计和核心模块开发：系统架构：采用前后端分离架构，前端用 Vue 3 + Element Plus + ECharts，后端使用 Spring Boot 提供 RESTful API，数据库采用 Neo4j 图数据库。知识图谱可视化：设计节点（如论文、作者、机构）和关系结构，使用 ECharts 力导向图实现动态渲染，支持分页加载优化性能。NLP模块：集成 Apache OpenNLP，实现英文实体识别和结构化数据抽取，自动生成 Cypher 语句导入 Neo4j。数据导入导出：支持Excel批量导入/导出，提升数据处理效率。智能问答接口：结合 NLP 与图数据库，实现语义化的知识问答功能。这个项目强化了我对图数据库、NLP与可视化技术融合的理解和实战能力。", "score": 8, "comments": "展现了对项目细节的深入理解，但部分技术点讲解